# Connect to the CARLA server

In [ ]:
import carla
import random
import time

# Prerequisite: CARLA must already be running in the background
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)  # seconds

world = client.get_world()
print("Connected to world:", world.get_map().name)

In [ ]:
# Display all available maps
available_maps = client.get_available_maps()
for i, map_name in enumerate(available_maps):
    print(f"{i+1}. {map_name}")

In [ ]:
# Warning: Changing the map takes a few seconds and resets the world
client.load_world('/Game/Carla/Maps/Town01')

In [ ]:
# Example: Spawn a Tesla Model 3 blueprint
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.find('vehicle.tesla.model3')
spawn_points = world.get_map().get_spawn_points()
vehicle = world.spawn_actor(vehicle_bp, random.choice(spawn_points))

> 💡 **Small Task:** Find the spawned vehicle on the map.  
> You can look around and move using the mouse and the WASD keys.

In [ ]:
# Set weather conditions
weather = carla.WeatherParameters.HardRainNight
world.set_weather(weather)

# Introduction to Pygame

In [ ]:
# Very Basic Pygame script that displays a blank white canvas
import pygame

# Initialize Pygame
pygame.init()

# Define window size
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("White Canvas in Pygame")

# Background color (white)
WHITE = (255, 255, 255)

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False  # Exit when the window is closed
    
    # Fill the screen with white
    screen.fill(WHITE)
    
    # Update the display
    pygame.display.flip()

# Quit Pygame
pygame.quit()

In [ ]:
# Displaying a Car and a Person

import pygame

# Initialize Pygame
pygame.init()

# Define window size
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Car and Person on Canvas")

# Background color (white)
WHITE = (255, 255, 255)

# Load images (example images of a car and a person)
car_img = pygame.image.load("media/car.png")
person_img = pygame.image.load("media/person.png")

# Optionally scale images to fit the window
car_img = pygame.transform.scale(car_img, (300, 240))
person_img = pygame.transform.scale(person_img, (300, 300))

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False  # Exit when the window is closed
        elif event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
            running = False  # Exit when ESC is pressed
    
    # Fill the screen with white
    screen.fill(WHITE)
    
    # Draw images on the canvas
    screen.blit(car_img, (200, 300))   # Car at position (200, 300)
    screen.blit(person_img, (500, 300)) # Person at position (500, 300)
    
    # Update the display
    pygame.display.flip()

# Quit Pygame
pygame.quit()


# Introduction to YOLO

In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model (nano version for fast performance)
model = YOLO('model/yolov8n.pt')
image_path = 'media/car.png'
image = cv2.imread(image_path)

# Run object detection
results = model(image)

# Draw bounding boxes and labels on the image
annotated_image = results[0].plot()

# Display the result
cv2.imshow('YOLOv8 Detection', annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
from ultralytics import YOLO

# Load YOLOv8 model (default is YOLOv8n = "nano" for good performance)
model = YOLO('model/yolov8n.pt')

# Open video stream (0 = webcam, or provide a file path)
cap = cv2.VideoCapture("media/hamilton_clip.mp4")

In [ ]:
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("No frame received. Stream ended.")
        break

    # Run YOLO inference
    results = model(frame, verbose=False)
    annotated_frame = results[0].plot()

    # Display the annotated frame
    cv2.imshow("YOLO Object Detection", annotated_frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Threaded Processing

In [ ]:
import threading
import time
import queue

# Background task that processes data from the input queue
def background_worker(input_queue, output_queue):
    while True:
        item = input_queue.get()
        if item is None:
            break  # Exit the thread if a termination signal is received
        # Simulate some processing (e.g., image analysis)
        processed = item.upper()  # Here, just convert text to uppercase
        time.sleep(0.5)  # Simulate processing time
        output_queue.put(processed)

# Queues for thread-safe communication between main thread and worker
input_queue = queue.Queue()
output_queue = queue.Queue()

# Start the background thread
thread = threading.Thread(target=background_worker,args=(input_queue, output_queue))
thread.start()

# Main thread: generate and send data to the worker
for i in range(5):
    text = f"message {i}"
    print(f"[Main] Sending: {text}")
    input_queue.put(text)
    time.sleep(0.2)  # Main thread is faster than the worker

# Retrieve results from the output queue
for _ in range(5):
    result = output_queue.get()
    print(f"[Main] Received: {result}")

# Send termination signal and wait for thread to finish
input_queue.put(None)
thread.join()
print("Thread finished.")


# TASK

In [3]:
# Interactive Pygame Example: Drag and Move Car and Person with the Mouse
import pygame

pygame.init()

# Define window size
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Car and Person on Canvas")

WHITE = (255, 255, 255)

# Load and scale images
car_img = pygame.image.load("media/car.png")
person_img = pygame.image.load("media/person.png")
car_img = pygame.transform.scale(car_img, (300, 240))
person_img = pygame.transform.scale(person_img, (300, 300))

# Starting positions of the images
car_pos = [200, 300]
person_pos = [500, 300]

# Dragging state flags
dragging_auto = False
dragging_person = False
offset_x = 0
offset_y = 0

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
            running = False

        elif event.type == pygame.MOUSEBUTTONDOWN:
            mouse_x, mouse_y = event.pos
            # Check if mouse is over the car
            if (car_pos[0] <= mouse_x <= car_pos[0] + car_img.get_width() and
                car_pos[1] <= mouse_y <= car_pos[1] + car_img.get_height()):
                dragging_auto = True
                offset_x = mouse_x - car_pos[0]
                offset_y = mouse_y - car_pos[1]
            # Or over the person
            elif (person_pos[0] <= mouse_x <= person_pos[0] + person_img.get_width() and
                  person_pos[1] <= mouse_y <= person_pos[1] + person_img.get_height()):
                dragging_person = True
                offset_x = mouse_x - person_pos[0]
                offset_y = mouse_y - person_pos[1]

        elif event.type == pygame.MOUSEBUTTONUP:
            dragging_auto = False
            dragging_person = False

        elif event.type == pygame.MOUSEMOTION:
            if dragging_auto:
                mouse_x, mouse_y = event.pos
                car_pos[0] = mouse_x - offset_x
                car_pos[1] = mouse_y - offset_y
            elif dragging_person:
                mouse_x, mouse_y = event.pos
                person_pos[0] = mouse_x - offset_x
                person_pos[1] = mouse_y - offset_y


        



                
    # Draw canvas and images 
    screen.fill(WHITE)
    screen.blit(car_img, car_pos)
    screen.blit(person_img, person_pos)
    pygame.display.flip()


surface_to_draw = pygame.Surface((WIDTH, HEIGHT))
view = pygame.surfarray.array3d(surface_to_draw)
# Convert to OpenCV format
view = view.transpose((1, 0, 2))  # Change from (width, height, channels) to (height, width, channes)
# Convert to BGR format
view = cv2.cvtColor(view, cv2.COLOR_RGB2BGR)
# Display the image using OpenCV
cv2.imshow("Carla View", view)
# Update the display
pygame.display.flip()




pygame.quit()

NameError: name 'cv2' is not defined

### 🧠 Task: Use OpenCV with YOLO on a Pygame Display

> Modify the Pygame script above to achieve the following:
- Capture the current Pygame display as an image
- Convert it to a format compatible with OpenCV
- Apply YOLO object detection to each frame
- Display the annotated result in real time using OpenCV